In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from ast import literal_eval
import reverse_geocode



In [2]:
prueba=pd.read_parquet(r'Datos limpios\restaurantes_yelps.parquet')

In [3]:
prueba

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa,FL,33602,27.955269,-82.456320,4.0,10,1,Fast Food
1,aPNXGTDkf-4bjhyMBQxqpQ,Craft Hall,901 N Delaware Ave,Center City,PA,19123,39.962582,-75.135657,3.5,65,1,American
2,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Carrollwood,FL,33618,28.046203,-82.505053,4.0,23,0,American
3,qfWJmJ0g96eM_fWma3ja0g,Doc Magrogan's Oyster House - West Chester,117 E Gay St,West Chester,PA,19380,39.961542,-75.603604,3.0,114,0,American
4,aCDY7vXYMs54EbYuQScsnQ,39 North Taproom & Grill,110 S MacDade Blvd,Glenolden,PA,19036,39.903697,-75.294981,4.5,25,1,American
...,...,...,...,...,...,...,...,...,...,...,...,...
12384,8MzF1Tlgz0pOkxmhP5dYzA,El Cap Restaurant,3500 4th St N,Lealman,FL,33704,27.804140,-82.638855,3.5,414,1,American
12385,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,900 Embarcadero Del Mar,Isla Vista,CA,93117,34.412966,-119.856886,2.5,12,1,American
12386,8n93L-ilMAsvwUatarykSg,Kitchen Gia,3716 Spruce St,Philadelphia,PA,19104,39.951018,-75.198240,3.0,22,0,American
12387,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Wharton,PA,19147,39.935982,-75.158665,4.5,35,1,Mexican


In [53]:
ruta_bussiness='Datasets/Yelp/business.pkl'

In [54]:
df_business = pd.read_pickle(ruta_bussiness)

In [55]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

In [56]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Eliminamos las columnas que en este caso estan repetidas pero antes verificamos que los datos sean repetidos

In [57]:
# Obtener los nombres de las columnas como una lista
column_names = df_business.columns.tolist()

# Encontrar el índice de la segunda columna 'business_id'
index = [i for i, name in enumerate(column_names) if name == 'business_id'][1]

# Cambiar el nombre de la segunda columna 'business_id'
column_names[index] = 'bussiness 2'

# Asignar los nuevos nombres de las columnas al DataFrame
df_business.columns = column_names

In [58]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'bussiness 2', 'name', 'address',
       'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars',
       'review_count', 'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

In [59]:
#Verificar que la segunda columna de business no es la misma que la otra
equal_values = (df_business['business_id'] == df_business['bussiness 2']).sum()

In [60]:
equal_values

5

Una vez verificado que las claves de los negocios son repetidas procedemos a eliminar dichas columnas.

In [61]:
df_business=df_business.iloc[:,:14]

In [62]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


Ahora procedemos a eliminar las columnas que no nos sirven para este proyecto.

In [63]:
df_business.drop(columns=['attributes'], inplace=True)

Si bien luego se realizara un análisis de la cantidad de nulos se eliminan las filas donde no tenemos datos acerca de la categoría.

In [64]:
#Eliminamos las filas de la columna 'categories' donde hay nulos
df_business.dropna(subset=['categories'], inplace=True)

In [65]:
df_business.isnull().sum()

business_id         0
name                0
address             0
city                0
state               3
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
categories          0
hours           23120
dtype: int64

In [66]:
df_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,IN,T6J 5H2,53.468419,-113.492054,3.0,13,1,"Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,DE,37204,36.115118,-86.766925,4.0,5,1,"Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,AB,46250,39.908707,-86.065088,3.5,8,1,"Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,AB,62025,38.782351,-89.950558,4.0,24,1,"Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


Comenzamos haciendo el filtro por resaurantes para luego proceder con los estados.

In [67]:
df=df_business.copy()

#Definimos las palabras claves
palabras_claves = ['restaurant', 'food','steakhouse','mexican','pizzeria','american','asian']

In [68]:
#Realizamos una función para convertir un string en una lista de palabras
def string_to_list(x):
    return [item.strip() for item in x.split(',')]

In [69]:
def filter_list_by_keywords(lst, palabras_claves):
    return [word for word in lst if any(palabra.lower() in word.lower() for palabra in palabras_claves)]


In [70]:
#Convertimos la columna 'categories' en una lista
df['categories'] = df['categories'].apply(string_to_list)

In [71]:
df['categories']

0         [Doctors, Traditional Chinese Medicine, Naturo...
1         [Shipping Centers, Local Services, Notaries, M...
2         [Department Stores, Shopping, Fashion, Home & ...
3         [Restaurants, Food, Bubble Tea, Coffee & Tea, ...
4                               [Brewpubs, Breweries, Food]
                                ...                        
150341                         [Nail Salons, Beauty & Spas]
150342    [Pets, Nurseries & Gardening, Pet Stores, Hobb...
150343    [Shopping, Jewelry, Piercing, Toy Stores, Beau...
150344    [Fitness/Exercise Equipment, Eyewear & Opticia...
150345    [Beauty & Spas, Permanent Makeup, Piercing, Ta...
Name: categories, Length: 150243, dtype: object

In [72]:
df['categories'] = df['categories'].apply(lambda x: filter_list_by_keywords(x, palabras_claves))

In [73]:
#Realizamos una funcion para extraer las palabras claves que no tengan que ver con el rubro
def filter_list_by_keywords(lst, palabras_claves):
    return [word for word in lst if any(palabra.lower() in word.lower() for palabra in palabras_claves)]

In [74]:
df=df.explode('categories')

In [75]:
df.dropna(subset=['categories'], inplace=True)


In [76]:
df['categories'] = df['categories'].apply(lambda x: x.title().strip())

In [77]:
df['categories'].value_counts()

categories
Restaurants               52268
Food                      27781
American (Traditional)     8139
Fast Food                  6472
American (New)             6097
Mexican                    4600
Specialty Food             4233
Seafood                    3539
Asian Fusion               1547
Steakhouses                1506
Food Trucks                1273
Food Delivery Services     1050
Latin American              810
Comfort Food                746
Ethnic Food                 658
Soul Food                   539
Food Stands                 423
Seafood Markets             337
Imported Food               201
Do-It-Yourself Food         145
Food Court                  130
New Mexican Cuisine         111
Live/Raw Food                88
Pop-Up Restaurants           75
Pan Asian                    69
Food Tours                   62
Restaurant Supplies          25
Food Banks                   12
Name: count, dtype: int64

In [78]:
df = df[df['categories'].apply(lambda x: len(x) > 0)]

In [80]:
df_dicc=pd.read_excel(r'E:\Data Science\PF\Diccionario Categorías.xlsx')

#hacemos un explode la columna categorias
df = df.explode('categories')
    
#Antes del merge usamos title  y strip para normalizar los campos
df['categories'] = df['categories'].apply(lambda x: x.title().strip())
df_dicc['ORIGINAL'] = df_dicc['ORIGINAL'].apply(lambda x: x.title().strip())

#Hacemos un merge con el diccionario de categorias
df=df.merge(df_dicc, left_on='categories', right_on='ORIGINAL', how='left')

#Eliminamos las columnas que no nos interesan
df.drop(columns=['ORIGINAL','categories'],inplace=True)

 #Renombramos la columna TRADUCCION
df.rename(columns={'TRADUCCION':'categories'},inplace=True)

#Eliminamos filas con valores nulos en la columna category
df.dropna(subset='categories',inplace=True)

In [81]:
df.reset_index(drop=True, inplace=True)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31094 entries, 0 to 31093
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   31094 non-null  object
 1   name          31094 non-null  object
 2   address       31094 non-null  object
 3   city          31094 non-null  object
 4   state         31094 non-null  object
 5   postal_code   31094 non-null  object
 6   latitude      31094 non-null  object
 7   longitude     31094 non-null  object
 8   stars         31094 non-null  object
 9   review_count  31094 non-null  object
 10  is_open       31094 non-null  object
 11  hours         27839 non-null  object
 12  categories    31094 non-null  object
dtypes: object(13)
memory usage: 3.1+ MB


In [83]:
#Limpiamos los strings 
df['name'] = df['name'].apply(lambda x: x.strip() if isinstance(x, str) else x)
df['address'] = df['address'].apply(lambda x: x.strip() if isinstance(x, str) else x)
df['city'] = df['city'].apply(lambda x: x.strip() if isinstance(x, str) else x)
df['state'] = df['state'].apply(lambda x: x.strip() if isinstance(x, str) else x)


Vamos a verificar la ciudad y el estado

In [84]:
#Creamos una funcion para verificar la ciudad y el estado
def get_location(row):
    coordinates = [(row['latitude'], row['longitude'])]
    location = reverse_geocode.search(coordinates)[0]
    if location['country_code'] == 'US':
        return pd.Series([location['city'], location['state']])
    else:
        return pd.Series([None, None])

df[['city', 'state']] = df.apply(get_location, axis=1)

In [85]:
df['state'].value_counts()

state
Pennsylvania    6146
Florida         5512
Tennessee       3098
Indiana         3025
Missouri        2749
Arizona         2003
Louisiana       1951
New Jersey      1547
Nevada          1168
Idaho            878
Illinois         806
California       731
Delaware         548
Name: count, dtype: int64

In [86]:
#Cambiamos los nombres de los estados por sus abreviaturas de los estados que nos interesan
df['state'] = df['state'].map({'Florida':'FL','Texas':'TX','California':'CA','New York':'NY','Pennsylvania':'PA'})

In [87]:
#Filtramos los restaurantes por estado
lista_estados=['FL','TX','CA','NY','PA']
df = df[df['state'].isin(lista_estados)]

In [88]:
df.isnull().sum()

business_id        0
name               0
address            0
city               0
state              0
postal_code        0
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
hours           1241
categories         0
dtype: int64

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12389 entries, 3 to 31093
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   12389 non-null  object
 1   name          12389 non-null  object
 2   address       12389 non-null  object
 3   city          12389 non-null  object
 4   state         12389 non-null  object
 5   postal_code   12389 non-null  object
 6   latitude      12389 non-null  object
 7   longitude     12389 non-null  object
 8   stars         12389 non-null  object
 9   review_count  12389 non-null  object
 10  is_open       12389 non-null  object
 11  hours         11148 non-null  object
 12  categories    12389 non-null  object
dtypes: object(13)
memory usage: 1.3+ MB


In [90]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,hours,categories
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa,FL,33602,27.955269,-82.45632,4.0,10,1,"{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",Fast Food
7,aPNXGTDkf-4bjhyMBQxqpQ,Craft Hall,901 N Delaware Ave,Center City,PA,19123,39.962582,-75.135657,3.5,65,1,"{'Monday': '0:0-0:0', 'Wednesday': '16:0-22:0'...",American
10,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Carrollwood,FL,33618,28.046203,-82.505053,4.0,23,0,"{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:...",American
13,qfWJmJ0g96eM_fWma3ja0g,Doc Magrogan's Oyster House - West Chester,117 E Gay St,West Chester,PA,19380,39.961542,-75.603604,3.0,114,0,None,American
16,aCDY7vXYMs54EbYuQScsnQ,39 North Taproom & Grill,110 S MacDade Blvd,Glenolden,PA,19036,39.903697,-75.294981,4.5,25,1,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-0:0', '...",American


In [91]:
#Se cambian los tipos de datos de latitude, logitude,stars,review_count
df = df.astype({
    'latitude': 'float64',
    'longitude': 'float64',
    'stars': 'float32',
    'review_count': 'int64'
})



In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12389 entries, 3 to 31093
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   12389 non-null  object 
 1   name          12389 non-null  object 
 2   address       12389 non-null  object 
 3   city          12389 non-null  object 
 4   state         12389 non-null  object 
 5   postal_code   12389 non-null  object 
 6   latitude      12389 non-null  float64
 7   longitude     12389 non-null  float64
 8   stars         12389 non-null  float32
 9   review_count  12389 non-null  int64  
 10  is_open       12389 non-null  object 
 11  hours         11148 non-null  object 
 12  categories    12389 non-null  object 
dtypes: float32(1), float64(2), int64(1), object(9)
memory usage: 1.3+ MB


In [93]:
df['state'].value_counts()

state
PA    6146
FL    5512
CA     731
Name: count, dtype: int64

In [94]:
#Borramos la columna hours que finalmente no se usará
df.drop(columns=['hours'], inplace=True)

In [95]:
df.to_parquet('Datos limpios/restaurantes_yelps.parquet', index=False)

In [97]:
df.isnull().sum()

business_id     0
name            0
address         0
city            0
state           0
postal_code     0
latitude        0
longitude       0
stars           0
review_count    0
is_open         0
categories      0
dtype: int64